In [2]:
import os
import pandas as pd
import numpy as np

import sys
sys.path.insert(0,'..')

import util


In [3]:

def load_data(data_dir: str, ds_set: str, lines: bool):

  ds_path = os.path.join(data_dir, ds_set + '.json')
  assert os.path.exists(ds_path), f"{ds_path} doesn't exsits"

  ds = pd.read_json(ds_path, lines=lines)
  ds['instructions'] = ds['content']
  return ds

In [4]:
 DATA_DIR = "/mnt/hackney/human_data"
 train = load_data(DATA_DIR, 'train', lines=True)
 valid = load_data(DATA_DIR, 'dev', lines=True)
 test = load_data(DATA_DIR, 'test', lines=True)


In [12]:
data_dir_geo = "/home/onlp_gcp_biu/cabby/app"

human_dataset = train
human_dataset['landmarks'] = ''


for human_dataset, data_type in zip([train, test, valid], ['train', 'dev', 'test']):
  geo_datasets_paths = list(set(human_dataset.rvs_path.tolist()))

  for geo_dataset_path in geo_datasets_paths:

    path_geo = geo_dataset_path.replace("/app_instructor", data_dir_geo)
    entities = util.load_entities(path_geo)

    keys = human_dataset[human_dataset['rvs_path']==geo_dataset_path].rvs_sample_number.tolist()
    entity_by_keys = np.array(entities)[keys]
    entity_landmarks_list = []
    for entity, key in zip(entity_by_keys, keys):
      landmarks = entity.geo_landmarks
      landmark_points = [f"{t}: {util.list_yx_from_point(v.geometry)}" for t, v in landmarks.items() if v.geometry]
      landmark_str = ';'.join(landmark_points)
      human_dataset.loc[human_dataset.rvs_sample_number== key, 'landmarks'] = landmark_str
      human_dataset.loc[human_dataset.rvs_sample_number== key, 'route'] = str(entity.route)



  new_dir = os.path.join(DATA_DIR, 'landmarks_data')
  if not os.path.exists(new_dir):
    os.mkdir(new_dir)
  new_path = os.path.join(new_dir, data_type + '.json')

  print (f'Saved {data_type} to: {new_path}')

  human_dataset.to_json(new_path, lines=True, orient='records')


LINESTRING (-73.9867199 40.7582685, -73.986679993796 40.75836247508798, -73.98677259999999 40.7584018, -73.98683283940527 40.75831994603107, -73.98687339387021 40.75826484017637, -73.9868782567749 40.75825823240778, -73.98695009796054 40.75816061381158, -73.98695565702576 40.7581530600924, -73.98714940000001 40.7578898, -73.98718909999999 40.7578359, -73.98884289999999 40.7585323, -73.98912152197865 40.75815076248908, -73.9892961 40.7579117, -73.98949404082107 40.75764066688703, -73.98965689474323 40.75741767697902, -73.989721 40.7573299, -73.9897914 40.7572335, -73.98985639999999 40.7571441, -73.99003257083172 40.75690185412317, -73.99022663771251 40.75663490412346, -73.99027599999999 40.756567, -73.99034593481123 40.75647127692061, -73.99036282342453 40.75644816067788, -73.99038112246231 40.75642311392055, -73.99053469834246 40.75621290735951, -73.99057590931774 40.7561564999511, -73.9907278 40.7559486, -73.9908244903034 40.75581542503198, -73.99083154095436 40.75580571392153, -73.99

KeyboardInterrupt: 